In [1]:
import unidecode 
import string
import random
import re
import random
import torch as T
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def preprocessing(filname='./pg1112.txt'):
    file = unidecode.unidecode(open(filname).read())
    data = [i for st in file for i in st]
    dico = defaultdict(int)
    for i, c in enumerate(set(data)):
        dico[c] = i + 1
    x_vect = [dico[c] for c in data]
    return x_vect, dico

In [3]:
x_vect, dico = preprocessing()

In [6]:
def make_batch(x_vect, batch_size=64, chunk_size=64): #FIXME
    idx_alea = random.randint(0,len(x_vect) - chunk_size)
    seq = T.tensor([x_vect[idx_alea - chunk_size : idx_alea]])
    for b in range(batch_size - 1):
        idx_alea = random.randint(0,len(x_vect) - chunk_size)
        new_seq = T.tensor([x_vect[idx_alea - chunk_size : idx_alea]])
        seq = T.cat((new_seq, seq), dim=0) #np.vstack((g, gg))
    seq = seq.permute(1, 0)
    return seq
batch_x = make_batch(x_vect)

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        #Gru a deux portes au lieu de 3 comme dans le LSTM
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input)
        output, hidden = self.gru(input, hidden) 
        output = self.decoder(output) 
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [8]:
len(dico) + 1

86

In [9]:
epochs = 1000
input_size = len(dico) + 1# dans le rnn ya un embedding (inputsize = taille dico => hidden size (taille de l'embdedding)) sinon la taille du dico +1
output_size = len(dico) + 1#nbr class =>softmax plus grd pas de bleme
hidden_size = 128 #arbitraire
n_layers = 1 # afixer
lr = 0.005

model = RNN(input_size, hidden_size, output_size, n_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

In [10]:
def train(data_x):
    hidden = model.init_hidden()
    optimizer.zero_grad()

    #chunk - 1
    loss = 0.
    for i, vect in enumerate(data_x[:-1]):
        output, hidden = model(vect.unsqueeze(0), hidden)
        loss += loss_function(output.squeeze(0), data_x[i+1])

    loss.backward()
    optimizer.step()

    return loss.item()


In [11]:
import time

start = time.time()
#all_losses = []
loss_avg = 0

for epoch in range(1, epochs + 1):
    try:
        loss = train(make_batch(x_vect))  #FIXME      


        if epoch % 100 == 0:
            print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / epochs * 100, loss))
            #print(evaluate('Wh', 100), '\n')
            generate(random.choice(dico.keys), 0.2)

        all_losses.append(loss)
    except:
        pass

[174.56721115112305 (100 10%) 155.9182]
[348.2949261665344 (200 20%) 153.1631]
[509.76208329200745 (300 30%) 159.2647]
[659.3681080341339 (400 40%) 155.4800]
[817.4218692779541 (500 50%) 154.1841]
[1450.374162197113 (600 60%) 156.2091]
[1604.5716562271118 (700 70%) 153.4763]
[1750.1680090427399 (800 80%) 153.3800]
[1761.2084822654724 (900 90%) 152.5264]
[1911.3740103244781 (1000 100%) 154.0321]


In [12]:
plt.plot(all_losses)

NameError: name 'all_losses' is not defined

In [13]:
def generate(caract='C', temp=1.):
    hidden = model.init_hidden()
    new_char = dico[caract]
    for i in range(200):
        predict, hidden = model(T.tensor([[new_char]]), hidden)
        softmax = T.nn.Softmax()(predict.view(86) / temp)
        new_char = T.multinomial(softmax, num_samples=1)
        print((['NULL'] + list(dico.keys()))[new_char], end='')


In [14]:
generate(0.2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


6],
  De; s hes y her ORorouly [aliserifil's  mbareorre t s.
 Prste s
  thimede horsllemy  S yoxiar.

  witono  ll- in houl,
 lanes He, dononde I  s  mye PLig me moks, thyinthet thopurisandinove h  ot